# Projeto 01 - Transcrição e compreensão de vídeos

Neste projeto, vamos aprender a realizar transcrição e compreensão de vídeos. Ao final, você será capaz de criar sua própria aplicação que faz a sumarização automática de vídeos, permitindo que você entenda do que se trata e o que foi falado nele sem precisar assistí-lo.

Objetivos deste projeto:

* Compreender o conteúdo de um vídeo do Youtube sem precisar assisti-lo.
* Pesquisar informações úteis no vídeo sem perder nenhum detalhe importante.
* Interagir com o conteúdo do vídeo por meio de uma interface de chat (ou seja, como "conversar com o vídeo").

## Instalação e Configuração

In [1]:
!pip install -q langchain_community langchain-huggingface langchain_ollama langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


### Instalação de bibliotecas para baixar transcrição

> **YouTube Transcript API**

Esta é uma API python que permite que você obtenha a transcrição/legendas para um determinado vídeo do YouTube. Ela também funciona para legendas geradas automaticamente e possui suporta a uma função que faz automaticamente a tradução de legendas

In [2]:
!pip install youtube-transcript-api

> **pytube**

Também é uma biblioteca que auxilia com o download de vídeos no youtube. Aqui ela não é necessária para baixar as transcrições dos vídeos, conseguimos ter acesso sem ela, mas iremos instalar também pois com ela podemos recuperar também demais informações do vídeo, como título, data de publicação, descrição, etc.

In [11]:
# !pip install pytube
!pip install --upgrade pytube

## Importações

In [12]:
import os
import io
import getpass
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.llms import HuggingFaceHub
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

## Carregando a transcrição

Para fazer o carregamento das transcrições usaremos o método YoutubeLoader(), que faz parte do dos document_loaders do LangChain. Veremos logo em seguida também como extrair os metadados do vídeo usando essa função

Através desses método conseguimos puxar as transcrições que já estão associadas ao vídeo e armazenadas no banco de dados do Youtube, o que irá nos economizar bastante processamento.


O primeiro parâmetro é a URL do vídeo que queremos realizar a transcrição

Vamos pegar esse vídeo como primeiro exemplo https://www.youtube.com/watch?v=II28i__Tf3M

### Defininido idiomas

O segundo parâmetro é o language. A função espera uma lista, nesse caso, uma lista de códigos de idioma em prioridade decrescente (por padrão).

Além de inglês ("en"), recomendamos deixar antes "pt" e "pt-BR" (ou "pt-PT") pois em alguns vídeos não possui "pt". Embora a grande maioria dos vídeos que testamos possua legenda com código "pt", mesmo para vídeos com a legenda em português brasileiro. Ou seja, deixamos assim pois em alguns vídeos do português brasileiro por exemplo o código é "pt", já para outros está como "pt-BR".


In [13]:
video_loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=II28i__Tf3M",
                                              language = ["pt", "pt-BR", "en"],)

Usaremos o .load() para fazer a leitura e ao mesmo tempo podemos passar as informações do vídeo para uma variável

In [14]:
infos = video_loader.load()
infos

[Document(metadata={'source': 'II28i__Tf3M'}, page_content='Olá sejam bem-vindos a sala que você vai aprender alguns fundamentos biológicos básicos sobre as redes neurais Primeiramente você verá sobre as redes neurais humana na sequência Vamos trabalhar com as redes neurais artificiais nós temos essa imagem representa os neurônios do cérebro existem bilhões de neurônios que estão conectados base nesta imagem nós vamos discutir três principais o primeiro ponto é que existem muitos neurônios o segundo ponto é que eles estão conectados entre si e o terceiro ponto é que estes neurônio por meio dessas conexões trocam informações entre si conexão entre os neurônios dos é e é responsável pelas nossas habilidades por exemplo ver falar dar e assim por diante vamos supor que você fala o idioma português e se indica que essas conexões entre os neurônios estão criadas de forma a permitir você falar esse idioma vamos supor que você começa a aprender a falar inglês com isso novas conexões Entre esse

O valor de "page_content" corresponde à transcrição em si

para acessá-la devemos colocar `[0]` pois `infos` é uma lista, que nesse caso só tem o primeiro valor. O código então fica assim

In [15]:
transcricao = infos[0].page_content
transcricao

'Olá sejam bem-vindos a sala que você vai aprender alguns fundamentos biológicos básicos sobre as redes neurais Primeiramente você verá sobre as redes neurais humana na sequência Vamos trabalhar com as redes neurais artificiais nós temos essa imagem representa os neurônios do cérebro existem bilhões de neurônios que estão conectados base nesta imagem nós vamos discutir três principais o primeiro ponto é que existem muitos neurônios o segundo ponto é que eles estão conectados entre si e o terceiro ponto é que estes neurônio por meio dessas conexões trocam informações entre si conexão entre os neurônios dos é e é responsável pelas nossas habilidades por exemplo ver falar dar e assim por diante vamos supor que você fala o idioma português e se indica que essas conexões entre os neurônios estão criadas de forma a permitir você falar esse idioma vamos supor que você começa a aprender a falar inglês com isso novas conexões Entre esses neurônios são geradas para permitir que você consiga fala

Esse primeiro exemplo é de uma legenda que foi gerada automaticamente pelo sistema de reconhecimento de fala do youtube, que no geral tende a ser bom mas pode gerar erros, então não é perfeito. Mas ainda assim, dependendo da LLM ela vai entender que se trata de um erro com base no contexto

Para legendas automáticas verificamos que não houve perda considerável na compreensão, mas obviamento é esperado que uma legenda feita manualmente possua maiores chances de resultados melhores

### Obter informações do vídeo

Note que carregamos a legenda/transcrição mas nenhuma outra informação sobre o vídeo, o que pode ser útil depedendo do nosso objetivo.

Podemos passar como parâmetro add_video_info=True (que por padrão é =False) e isso fará com que sejam retornados os metadados do vídeo, como: título, descrição, autor, visualizações, e capa)

Para usar esse parâmetro você precisa ter instalado antes a biblioteca pytube

In [53]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 57.1 MB/s eta 0:00:00


In [105]:
def formatted_date(data):
    year = data[0:4]
    month = data[4:6]
    day = data[6:8]
    formatted_date = f"{day}/{month}/{year}"
    return formatted_date

In [168]:
import yt_dlp

youtube_url = "https://www.youtube.com/watch?v=II28i__Tf3M"

ydl_opts = {}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    video_info = ydl.extract_info(youtube_url, download=False)
    # title = video_info.get('title', 'Título indisponível')
    # print(f"Título: {title}")
    # print("Informações do vídeo:", video_info)

[youtube] Extracting URL: https://www.youtube.com/watch?v=II28i__Tf3M
[youtube] II28i__Tf3M: Downloading webpage
[youtube] II28i__Tf3M: Downloading ios player API JSON
[youtube] II28i__Tf3M: Downloading mweb player API JSON
[youtube] II28i__Tf3M: Downloading m3u8 information


In [169]:
infos_video = f"""Informações do vídeo:

Título: {video_info.get('title', 'Título indisponível')}
Autor: {video_info.get('uploader', 'Autor indisponível')}
Data: {formatted_date(video_info['upload_date'])}
URL: {video_info['channel_url']}
Transcrição: {video_info["description"]}
"""
print(infos_video)

Informações do vídeo:

Título: Introdução aos fundamentos biológicos das redes neurais artificiais
Autor: IA Expert Academy
Data: 29/03/2022
URL: https://www.youtube.com/channel/UCaGrIWpwjWXT6OIQh9W4Riw
Transcrição: Nessa videoaula você será apresentado a noções introdutórias sobre os fundamentos biológicos das redes neurais artificiais. A aula faz parte do curso "Algoritmos de Inteligência Artificial Bioinspirados" que está disponível para assinantes IA Expert Academy. 

► LINKS
Detalhes do curso: https://iaexpert.academy/courses/algoritmos-de-inteligencia-artificial-bioinspirados/

Todos os nossos cursos: https://iaexpert.academy/cursos-assinatura/

Saiba como se tornar um assinante da nossa plataforma de cursos online sobre inteligência artificial: https://iaexpert.academy/assinatura/

Também é possível realizar o curso através da plataforma Udemy: https://www.udemy.com/course/algoritmos-de-inteligencia-artificial-bioinspirados/?referralCode=74F83D8343C970A03B2C

Faça download do no

In [167]:
# video_loader = YoutubeLoader.from_youtube_url(youtube_url="https://www.youtube.com/watch?v=II28i__Tf3M",
#                                                 add_video_info = True,
#                                                language = ["pt", "pt-BR", "en"],)
# # infos = video_loader.load()
# # infos

Podemos organizar desse modo

In [173]:
# infos_video = f"""Informações do vídeo:

# Título: {infos[0].metadata['title']}
# Autor: {infos[0].metadata['author']}
# Data: {infos[0].metadata['publish_date'][:10]}
# URL: https://www.youtube.com/watch?v={infos[0].metadata['source']}

# Transcrição: {transcricao}
# """
# print(infos_video)

## Salvando transcrição em um arquivo

Esse código abre um arquivo chamado "transcricao.txt" em modo de escrita ("w") com codificação UTF-8;  dentro do bloco `with` ele grava dados no arquivo.

Para cada item na variável `infos`, ele escreve o conteúdo da variável `infos_video` no arquivo. O uso do bloco with garante que o arquivo seja fechado corretamente após a gravação, mesmo que ocorra algum erro durante a execução. E aqui não precisa do `f.close()` pois o bloco with fecha o arquivo automaticamente ao finalizar

In [119]:
with io.open("transcricao.txt", "w", encoding="utf-8") as f:
  for doc in infos:
    f.write(infos_video)

## Carregamento do modelo

Vamos reaproveitar as funções de carregamento que usamos nos projeos anteriores, basta copiar e colar

In [120]:
def model_hf_hub(model = "meta-llama/Meta-Llama-3-8B-Instruct", temperature = 0.1):
  llm = HuggingFaceHub(repo_id = model,
                       model_kwargs={
                           "temperature": temperature,
                           "return_full_text": False,
                           "max_new_tokens": 1024,
                       })
  return llm

In [ ]:
def model_openai(model = "gpt-4o-mini", temperature = 0.1):
  llm = ChatOpenAI(model = model, temperature = temperature)
  return llm

In [ ]:
def model_ollama(model = "phi3", temperature = 0.1):
  llm = ChatOllama(model = model, temperature = temperature)
  return llm

E aqui no Colab precisar setar as variáveis de ambiente. Pode usar o .env também, especialmente se estiver executando localmente

In [121]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass()

··········


In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [122]:
model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]

if model_class == "hf_hub":
  llm = model_hf_hub()
elif model_class == "openai":
  llm = model_openai
elif model_class == "ollama":
  llm = model_ollama


<ipython-input-120-7c8108b6e615>:2: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(repo_id = model,


In [123]:
model_class, llm

('hf_hub',
 HuggingFaceHub(client=<InferenceClient(model='meta-llama/Meta-Llama-3-8B-Instruct', timeout=None)>, repo_id='meta-llama/Meta-Llama-3-8B-Instruct', task='text-generation', model_kwargs={'temperature': 0.1, 'return_full_text': False, 'max_new_tokens': 1024}))

## Criação do prompt template

Vamos manter a base do prompt simples, basicamente instruindo que deve responder com base na transcrição fornecida. Você pode modificá-lo à vontade depois, para deixar mais adequado ao seu objetivo ou simplesmente para tentar alcançar melhores resultados

* Aqui vamos passar o transcrição completa. Estaremos lidando com modelos que possuem uma janela grande de contexto - por exemplo o llama 3 possui algo em torno de 8k, já o chatGPT 4o por exemplo possui ainda mais. Deve ser uma capacidade suficiente de leitura de tokens de entrada para lidar com a maioria das transcrições dos vídeos, e será para todos os testados aqui.
* Como a ideia desse projeto é criar uma ferramenta que faz o resumo / sumarização então adicionar a transcrição inteira como contexto é até uma opção mais interessante, já que para RAG é recuperado geralmente uma quantidade limite de pedaços de documento. Portanto, se fosse usado RAG teria que configurar bem os parâmetros, provavelmente escolher um valor maior de k por exemplo para recuperar mais documentos (no entanto, lembre-se que elevar muito esse valor aumenta o custo computacional da aplicação)
* Mas caso o vídeo seja realmente grande então pode ser interessante dividir em partes. Para isso sugerimos usar o código do projeto 3, pode copiar as funções prontas que fazem as etapas de indexação e recuperação (indexing & retrieval)

Além da transcrição, o prompt template irá aceitar a variável consulta, que nada mais é do que a entrada para a LLM, que pode ser uma pergunta ou instrução

E o `if model_class.startswith("hf"):` apenas copiamos do projeto anterior, lembrando que isso é para melhorar os resultados com a implementação via Hugging Face Hub, que até o momento funciona melhor se especificarmos manualmente os tokens de início e fim. Aqui o template é do llama 3.x, mas for usar outro modelo open source que exija template diferente então lembre de mudar.


In [124]:
system_prompt = "Você é um assistente virtual prestativo e deve responder a uma consulta com base na transcrição de um vídeo, que será fornecida abaixo."

inputs = "Consulta: {consulta} \n Transcrição: {transcricao}"

if model_class.startswith("hf"):
  user_prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>".format(inputs)
else:
  user_prompt = "{}".format(inputs)

prompt_template = ChatPromptTemplate.from_messages([("system", system_prompt), ("user", user_prompt)])

In [125]:
prompt_template

ChatPromptTemplate(input_variables=['consulta', 'transcricao'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Você é um assistente virtual prestativo e deve responder a uma consulta com base na transcrição de um vídeo, que será fornecida abaixo.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['consulta', 'transcricao'], input_types={}, partial_variables={}, template='<|begin_of_text|><|start_header_id|>user<|end_header_id|>\nConsulta: {consulta} \n Transcrição: {transcricao}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'), additional_kwargs={})])

## Criação da chain

Nossa chain ficará assim

In [126]:
chain = prompt_template | llm | StrOutputParser()

## Geração da resposta

Por fim, vamos gerar o resultado, fornecendo como parâmetro a transcrição e a consulta que queremos (podendo ser pergunta, instrução, etc.)

In [127]:
res = chain.invoke({"transcricao": transcricao, "consulta": "resuma"})
print(res)



Aqui está um resumo da transcrição:

A aula começa apresentando a estrutura do cérebro humano, com bilhões de neurônios conectados entre si. Os neurônios são responsáveis pelas habilidades humanas, como falar, ver e ouvir. A troca de informações entre os neurônios é feita por meio de conexões, chamadas sinapses.

O neurônio é composto por três principais componentes: dendritos, corpo celular e axônio. Os dendritos recebem informações, o corpo celular processa a informação e o axônio transmite a informação processada para o próximo neurônio.

A rede neural é formada pela conexão entre vários neurônios, onde a troca de informações ocorre por meio da sinapse. A sinapse é representada por processos elétricos no cérebro humano, onde o potencial elétrico do corpo celular do neurônio é alterado quando uma informação é passada de um neurônio para outro.

A aula conclui apresentando os principais conceitos biológicos sobre as redes neurais, preparando o aluno para entender como funcionam os n

Podemos melhorar esse prompt (consulta), deixando algo como `"sumarize de forma clara de entender`

In [128]:
res = chain.invoke({"transcricao": transcricao, "consulta": "sumarize de forma clara de entender"})
print(res)



Aqui está uma resumo claro e fácil de entender da transcrição:

As redes neurais são compostas por bilhões de neurônios que se conectam entre si e trocam informações. Existem três principais pontos:

1. Existem muitos neurônios.
2. Eles estão conectados entre si.
3. Essas conexões permitem que os neurônios troquem informações entre si.

Essas conexões são responsáveis pelas nossas habilidades, como falar, ver e ouvir. Quando aprendemos uma habilidade nova, novas conexões são criadas entre os neurônios para permitir que aprendamos.

Um neurônio é composto por:

* Dendritos: recebem informações
* Corpo celular: processa a informação
* Axónio: transmite a informação processada para outro neurônio

A troca de informações entre neurônios é chamada de sinapse e ocorre quando o potencial elétrico do corpo celular do neurônio é alterado.

Em resumo, as redes neurais são compostas por neurônios que se conectam e trocam informações, permitindo que aprendamos e desenvolvamos habilidades.


In [129]:
res = chain.invoke({"transcricao": transcricao, "consulta": "explique em 1 frase sobre o que fala esse vídeo"})
print(res)



O vídeo explica os fundamentos biológicos das redes neurais, destacando a estrutura e funcionamento dos neurônios humanos e a troca de informações entre eles.


In [130]:
res = chain.invoke({"transcricao": transcricao, "consulta": "liste os temas desse video"})
print(res)



Aqui estão os temas do vídeo:

1. Redes neurais humanas
2. Componentes do neurônio (dendritos, corpo celular, axónio, terminais do axónio)
3. Sinapse (troca de informações entre neurônios)
4. Processamento de informações nos neurônios
5. Redes neurais artificiais
6. Aprendizado e formação de conexões entre neurônios
7. Função do cérebro humano em relação às redes neurais.


## Tradução da transcrição

Para os modelos mais modernos não é necessário traduzir antes, pode carregar a transcrição no idioma desejado e passar para a LLM mesmo que no idioma diferente daquele que você escreveu as instruções no prompt template, isso porque o modelo deve ser capaz de entender.

Mas também é possível traduzir a transcrição usando essa mesma ferramenta.
Isso pode ser muito útil caso o modelo que esteja trabalhando não funcione bem para múltiplos idiomas.

Para implementar isso, basta definirmos para o parâmetro translation o código do idioma para o qual desejamos traduzir.
Por exemplo para o francês ficaria assim

In [ ]:
# url_video = "https://www.youtube.com/watch?v=II28i__Tf3M"

# video_loader = YoutubeLoader.from_youtube_url(
#     url_video,
#     add_video_info=True,
#     language=["pt", "en"],
#     translation="fr",
# )

# infos = video_loader.load()
# transcricao = infos[0].page_content
# transcricao

## Junção da pipeline em funções

Para deixar mais prático e evitar repetições do código vamos reunir toda a nossa lógica em funções, assim não vai ser mais necessário ficar copiando e colando o código toda vez que for testar em outro vídeo

In [170]:
def llm_chain(model_class):
  system_prompt = "Você é um assistente virtual prestativo e deve responder a uma consulta com base na transcrição de um vídeo, que será fornecida abaixo."

  inputs = "Consulta: {consulta} \n Transcrição: {transcricao}"

  if model_class.startswith("hf"):
      user_prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>".format(inputs)
  else:
      user_prompt = "{}".format(inputs)

  prompt_template = ChatPromptTemplate.from_messages([
      ("system", system_prompt),
      ("user", user_prompt)
  ])

  ### Carregamento da LLM
  if model_class == "hf_hub":
      llm = model_hf_hub()
  elif model_class == "openai":
      llm = model_openai()
  elif model_class == "ollama":
      llm = model_ollama()

  chain = prompt_template | llm | StrOutputParser()

  return chain

In [171]:
def get_video_info(url_video, language="pt", translation=None):

  video_loader = YoutubeLoader.from_youtube_url(
      url_video,
      add_video_info=True,
      language=language,
      translation=translation,
  )

  infos = video_loader.load()[0]
  metadata = infos.metadata
  transcript = infos.page_content

  return transcript, metadata

In [209]:
def get_transcription(url_video):
  ydl_opts = {}
  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      video_info = ydl.extract_info(url_video, download=False,
                                    )
  transcript = video_info["description"]
  return transcript

Vamos testar abaixo

In [210]:
# transcript, metadata = get_video_info("https://www.youtube.com/watch?v=II28i__Tf3M")
transcript = get_transcription("https://www.youtube.com/watch?v=II28i__Tf3M")
transcript

[youtube] Extracting URL: https://www.youtube.com/watch?v=II28i__Tf3M
[youtube] II28i__Tf3M: Downloading webpage
[youtube] II28i__Tf3M: Downloading ios player API JSON
[youtube] II28i__Tf3M: Downloading mweb player API JSON
[youtube] II28i__Tf3M: Downloading m3u8 information


'Nessa videoaula você será apresentado a noções introdutórias sobre os fundamentos biológicos das redes neurais artificiais. A aula faz parte do curso "Algoritmos de Inteligência Artificial Bioinspirados" que está disponível para assinantes IA Expert Academy. \n\n► LINKS\nDetalhes do curso: https://iaexpert.academy/courses/algoritmos-de-inteligencia-artificial-bioinspirados/\n\nTodos os nossos cursos: https://iaexpert.academy/cursos-assinatura/\n\nSaiba como se tornar um assinante da nossa plataforma de cursos online sobre inteligência artificial: https://iaexpert.academy/assinatura/\n\nTambém é possível realizar o curso através da plataforma Udemy: https://www.udemy.com/course/algoritmos-de-inteligencia-artificial-bioinspirados/?referralCode=74F83D8343C970A03B2C\n\nFaça download do nosso e-book gratuito com 21 dicas de carreira em Inteligência Artificial e Machine Learning: https://iaexpert.academy/ebook-21-dicas-de-carreira-em-inteligencia-artificial-machine-learning/\n\nNão se esque

In [208]:
# metadata, transcript

Aqui aproveitamos para adicionar um Tratamento de erro com Try Catch, pois caso não haja uma transcrição para esse vídeo será retornado um erro (possivelmente será esse: `IndexError: list index out of range`), com isso não será possível fazer os processamentos seguintes. Por isso programos aqui para o programa parar interromper a execuão se esse for caso

In [211]:
def interpret_video(url, query="resuma", model_class="hf_hub", language="pt", translation=None):

  try:
    transcript = get_transcription(url_video)
    # transcript, metadata = get_video_info(url, language, translation)

    chain = llm_chain(model_class)

    res = chain.invoke({"transcricao": transcript, "consulta": query})
    print(res)

  except Exception as e:
    print("Erro ao carregar transcrição")
    print(e)

## Geração final

Podemos definir uma interface mais apresentável no Colab através dos comandos para deixar as variáveis em formato de valores de formulário.

Ideias do que adicionar à query:

* `sumarize de forma clara de entender`
* `liste os temas desse vídeo`
* `explique em 1 frase sobre o que fala esse vídeo`

In [212]:
url_video = "https://www.youtube.com/watch?v=II28i__Tf3M" # @param {type:"string"}
query_user = "sumarize de forma clara de entender" # @param {type:"string"}
model_class = "openai" # @param ["hf_hub", "openai", "ollama"]
language = ["pt", "pt-BR", "en"] # @param {type:"string"}

In [213]:
interpret_video(url_video, query_user, model_class, language)

[youtube] Extracting URL: https://www.youtube.com/watch?v=II28i__Tf3M
[youtube] II28i__Tf3M: Downloading webpage
[youtube] II28i__Tf3M: Downloading ios player API JSON
[youtube] II28i__Tf3M: Downloading mweb player API JSON
[youtube] II28i__Tf3M: Downloading m3u8 information
Erro ao carregar transcrição
name 'model_openai' is not defined


In [214]:
url_video = "https://www.youtube.com/watch?v=rEE8ERGKsqo" # @param {type:"string"}
query_user = "sumarize de forma clara de entender" # @param {type:"string"}
model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]
language = ["pt", "pt-BR", "en"] # @param {type:"string"}

interpret_video(url_video, query_user, model_class, language)

[youtube] Extracting URL: https://www.youtube.com/watch?v=rEE8ERGKsqo
[youtube] rEE8ERGKsqo: Downloading webpage
[youtube] rEE8ERGKsqo: Downloading ios player API JSON
[youtube] rEE8ERGKsqo: Downloading mweb player API JSON
[youtube] rEE8ERGKsqo: Downloading m3u8 information


Resumo:

O vídeo é sobre astronomia e visa ajudar aqueles que gostam do assunto mas se sentem perdidos ao ler sobre ele. O conteúdo do vídeo é destinado a ensinar o básico da ciência dos astros e é recomendado que os espectadores preparem prints e cadernos de anotações para acompanhar o vídeo.


## Explorando mais

Vamos deixar nossa aplicação mais interessante. Podemos fazer mais de uma consulta/query de uma vez por vídeo.

* Para cada vídeo informado, podemos definir para exibir informações como o titulo dele;
* e logo abaixo um resumo em um paragrafo;
* depois uma lista de temas abordados, etc.

No Colab temos um modo interessante de fazer isso, que é através do Markdown.


### Exemplo com Markdown

Markdown é um formato de linguagem de marcação muito adotada pela comunidade por ser simples e leve, permitindo criar texto formatado com uso de símbolos especiais, como asteriscos e colchetes, em vez de tags de HTML. No Google Colab, o uso de markdown pode tornar a visualização do texto mais interessante e fácil de ler, facilitando a compreensão e a apresentação de informações. Por exemplo, usando markdown, você pode deixar um texto em *itálico* ao deixar dentro de asteriscos, ou **negrito** se deixar ele dentro de asteriscos duplos. Também podemos adicionar títulos com diferentes níveis, por exemplo para nível 1,2,3 basta colocar antes da frase `#` `##` ou `###` respectivamente

> Mais sobre a sintaxe aqui: https://www.markdownguide.org/basic-syntax/

In [216]:
texto = """
### Título
descrição em **destaque** *aqui...*
"""

In [217]:
from IPython.display import Markdown

display(Markdown(texto))


### Título
descrição em **destaque** *aqui...*


E abaixo em forma de lista por exemplo

Com isso, se pedirmos por exemplo pra LLM retornar uma lista então ficará mais apresentável também, já que ela irá retornar nesse formato com * no inicio de cada item, já que é um padrão comum

Nos projetos em que usamos os Streamlit talvez tenha notada que já ficou automaticamente dessem modo, pois a interface já interpreta corretamente markdown

In [218]:
lista = """
* item
* item
* item
"""

In [219]:
display(Markdown(lista))


* item
* item
* item


### Finalização do código

Copiamos do código que fizemos antes, só mudando de infos[0].metadata para metadata

e ao invés do print() normal, vamos colocar no lugar display(Markdown()) para deixar mais apresentável

Como resposta, teremos:
* Informações do vídeo - que são os metadados: título, autor, data, URL.

*  Sobre o que fala o vídeo - resumo de 1 frase, para contextualizarmos bem rapidamente

* Temas - listagem dos principais temas desse vídeo

* Resposta para a consulta - que é a resposta para a consulta personalizada que fizemos, que pode ser uma pergunta ou instrução

Aqui você pode customizar à vontade depois, deixar com as consultas que achar mais conveniente para o objetivo de sua aplicação

Você poderia também separar em duas funções: uma para exibir junto as consultas fixas (informações do vídeo, resumo e temas) e outra para exibir a consulta personalizada, podendo deixar em formato de chat, igual feito no projeto 1 e 2.

mas aqui estamos deixando mais simples e rápido, portanto ficou desse modo

In [224]:
def interpret_video(url, query="liste os temas desse vídeo", model_class="hf_hub", language="pt", translation=None):

  try:
    # transcript, metadata = get_video_info(url, language, translation)

    ydl_opts = {}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      video_info = ydl.extract_info(url_video, download=False)

    transcript = video_info["description"]

    infos_video = f"""## Informações do vídeo

    Título: {video_info.get('title', 'Título indisponível')}
    Autor: {video_info.get('uploader', 'Autor indisponível')}
    Data: {formatted_date(video_info['upload_date'])}
    URL: {video_info['channel_url']}
    """

    display(Markdown(infos_video))

    chain = llm_chain(model_class)

    t = "\n## Sobre o que fala o vídeo \n"
    res = chain.invoke({"transcricao": transcript, "consulta": "explique em 1 frase sobre o que fala esse vídeo. responda direto com a frase"})
    display(Markdown(t + res))

    t = "\n## Temas \n"
    res = chain.invoke({"transcricao": transcript, "consulta": "lista os principais temas desse vídeo"})
    display(Markdown(t + res))

    t = "\n## Resposta para a consulta \n"
    res = chain.invoke({"transcricao": transcript, "consulta": query})
    display(Markdown(t + res))

  except Exception as e:
    print("Erro ao carregar transcrição")
    print(e)

In [225]:
url_video = "https://www.youtube.com/watch?v=rEE8ERGKsqo" # @param {type:"string"}
query_user = "sumarize de forma clara de entender" # @param {type:"string"}
model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]
language = ["pt", "pt-BR", "en"] # @param {type:"string"}

interpret_video(url_video, query_user, model_class, language)

[youtube] Extracting URL: https://www.youtube.com/watch?v=rEE8ERGKsqo
[youtube] rEE8ERGKsqo: Downloading webpage
[youtube] rEE8ERGKsqo: Downloading ios player API JSON
[youtube] rEE8ERGKsqo: Downloading mweb player API JSON
[youtube] rEE8ERGKsqo: Downloading m3u8 information


## Informações do vídeo

    Título: TUDO QUE VOCÊ PRECISA SABER PRA ENTENDER O BÁSICO SOBRE ASTRONOMIA
    Autor: Universo Interessado
    Data: 26/03/2023
    URL: https://www.youtube.com/channel/UCtJAiOB45Wil79b4zESqULg
    


## Sobre o que fala o vídeo 


O vídeo é sobre como aprender os fundamentos da astronomia de forma fácil e prática.


## Temas 


Os principais temas desse vídeo são:

1. Astronomia
2. Ciência dos astros
3. Entender o básico da astronomia


## Resposta para a consulta 
assistant

In [226]:
url_video = "https://www.youtube.com/watch?v=n9u-TITxwoM" # @param {type:"string"}
query_user = "sumarize de forma clara de entender" # @param {type:"string"}
model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]
language = ["pt", "pt-BR", "en"] # @param {type:"string"}

interpret_video(url_video, query_user, model_class, language)

[youtube] Extracting URL: https://www.youtube.com/watch?v=n9u-TITxwoM
[youtube] n9u-TITxwoM: Downloading webpage
[youtube] n9u-TITxwoM: Downloading ios player API JSON
[youtube] n9u-TITxwoM: Downloading mweb player API JSON
[youtube] n9u-TITxwoM: Downloading m3u8 information


## Informações do vídeo

    Título: The six degrees | Kevin Bacon | TEDxMidwest
    Autor: TEDx Talks
    Data: 28/06/2012
    URL: https://www.youtube.com/channel/UCsT0YIqwnpJCM-mx7-gSA4Q
    


## Sobre o que fala o vídeo 


O vídeo fala sobre a carreira e filantropia do ator Kevin Bacon e a fundação do TEDx, um programa de eventos que promove a discussão e conexão entre pessoas.


## Temas 


Os principais temas desse vídeo são:

1. Kevin Bacon
2. Cinema
3. Cultura popular
4. Caridade
5. SixDegrees.org
6. TEDx
7. Eventos de palestras
8. Comunicação e conexão entre pessoas


## Resposta para a consulta 


Resumo:

Kevin Bacon é um ator que participou de filmes influentes e icônicos. Ele criou a SixDegrees.org, uma iniciativa filantrópica que conecta pessoas a organizações de caridade e entre si para fazer uma diferença. O TEDx é um programa que organiza eventos locais e autogerenciados que trazem pessoas juntas para compartilhar experiências TED-like e estimular discussões e conexões profundas. Os eventos TEDx são autogerenciados, mas estão sujeitos a certas regras e regulamentações.

In [227]:
url_video = "https://www.youtube.com/watch?v=XXcHDy3QH-E" # @param {type:"string"}
query_user = "sumarize de forma clara de entender" # @param {type:"string"}
model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]
language = ["pt", "pt-BR", "en"] # @param {type:"string"}

interpret_video(url_video, query_user, model_class, language)

[youtube] Extracting URL: https://www.youtube.com/watch?v=XXcHDy3QH-E
[youtube] XXcHDy3QH-E: Downloading webpage
[youtube] XXcHDy3QH-E: Downloading ios player API JSON
[youtube] XXcHDy3QH-E: Downloading mweb player API JSON
[youtube] XXcHDy3QH-E: Downloading m3u8 information


## Informações do vídeo

    Título: Como teoria do caos e efeito borboleta ajudam a explicar Universo
    Autor: BBC News Brasil
    Data: 21/08/2022
    URL: https://www.youtube.com/channel/UCthbIFAxbXTTQEC7EcQvP1Q
    


## Sobre o que fala o vídeo 


A teoria do caos questiona a visão determinista de que o comportamento de um objeto pode ser previsto com facilidade, introduzindo um elemento de incerteza na nossa compreensão do Universo.


## Temas 


Os principais temas desse vídeo são:

1. Teoria do caos
2. Determinismo e incerteza
3. Física clássica e leis de Newton
4. Previsibilidade e comportamento dos objetos
5. Influência da teoria do caos na vida cotidiana


## Resposta para a consulta 


Resumo:

A teoria do caos questiona a visão determinista de que, com conhecimento do estado atual de um objeto, podemos prever com facilidade seu comportamento futuro. Isso significa que nem tudo é previsível e que o comportamento do universo não é sempre linear e previsível, como era pensado anteriormente. A teoria do caos não é a mesma coisa que desordem, mas sim um conceito que introduz incerteza na nossa compreensão do universo.

## Reconhecimento de fala

Se o vídeo não tiver uma transcrição disponível, será necessário gerar uma de forma automática utilizando um modelo de reconhecimento de fala, também conhecido como Speech-to-Text (STT). Esses modelos convertem fala em texto a partir de arquivos de áudio, permitindo que o conteúdo do vídeo seja transcrito para ser processado pela nossa aplicação de sumarização.

Um exemplo popular de modelo de Speech-to-Text é o [Whisper](https://openai.com/index/whisper/), da OpenAI, que oferece uma solução robusta para transcrição automática. Os preços e detalhes sobre o uso desse modelo podem ser encontrados na página de "[pricing](https://openai.com/api/pricing/)" da OpenAI. No entanto, neste projeto, optamos por não utilizá-lo, pois todos os vídeos testados já possuíam transcrições, muitas vezes geradas automaticamente pelo YouTube. Essa abordagem economiza processamento, já que o foco principal foi a implementação de modelos de linguagem grandes (LLMs) e a exploração de ferramentas do Langchain.

* Se você precisar integrar um serviço de transcrição, a implementação é simples, podendo ser feita com uma chamada de função. A documentação do Langchain oferece instruções detalhadas sobre como realizar essa integração aqui: https://js.langchain.com/v0.2/docs/integrations/document_loaders/file_loaders/openai_whisper_audio/.

* Na verdade você tem a liberdade de escolher qualquer serviço ou código para realizar o reconhecimento de fala. Mesmo que o serviço não se integre diretamente ao Langchain, isso não é um problema, pois o essencial é obter o texto final. No final, o que você obtém é um texto normal, que pode ser armazenado em um arquivo de texto ou diretamente em uma variável.

* Entretanto, se desejar aproveitar soluções prontas dentro do Langchain, você pode utilizar o Whisper da OpenAI ou outras opções como o [GoogleSpeechToText](https://python.langchain.com/v0.2/docs/integrations/document_loaders/google_speech_to_text/) e o [AssemblyAI Audio Transcripts](https://python.langchain.com/v0.2/docs/integrations/document_loaders/assemblyai/), que também são altamente eficazes e fáceis de integrar ao langchain.

## Indo além 🚀

1. Conforme mencionado anteriormente, para vídeos muito longos ou transcrições maiores, pode ser necessário dividir a transcrição em partes menores e aplicar técnicas de indexação e recuperação utilizando RAG.
 * Isso é essencial caso o vídeo que deseja processar possua horas por exemplo - talvez modelos muito modernos e com janela de contexto maior consigam lidar, mas mesmo que consigam pode ser interessante aplicar essas técnicas, principalmente caso o que busque não seja a sumarização mas sim algo próximo do nosso projeto de perguntas e respostas com documentos, que será visto no projeto 3.
 * Caso deseje seguir esse caminho, basta reutilizar o código já pronto no projeto 3, que lida com esse tipo de segmentação e recuperação de forma eficiente. Assim, você poderá manter a integridade da transcrição, otimizando o processamento e a geração de respostas mais precisas, mesmo com grandes volumes de dados.

2. Como ideia adicional ou desafio: é possível personalizar este projeto para atender a outras necessidades, ou ainda usar a sumarização como uma etapa posterior dentro de uma pipeline maior.
  * Um exemplo seria o uso da ferramenta [YouTubeSearchTool](https://python.langchain.com/v0.2/docs/integrations/tools/youtube//) do Langchain, que permite buscar automaticamente uma lista de URLs de vídeos no YouTube com base em um tema ou palavra-chave fornecida pelo usuário.
  * Nessa abordagem, você poderia implementar uma aplicação que solicita um termo de busca e, em seguida, utiliza essa ferramenta para buscar vídeos relevantes, retornando em uma lista. Por fim, basta criar um laço de repetição que execute a função interpret_video (que criamos nesse projeto) para cada vídeo da lista, realizando assim a sumarização de múltiplos vídeos associados a um tema de forma automatizada.

3. Além disso, é possível integrar essa funcionalidade a uma interface interativa com ferramentas como Streamlit, que permite criar facilmente interfaces gráficas.
 * Os métodos para essa integração estão descritos detalhadamente nos projetos 2 e 3, caso queira expandir a aplicação.
 * Embora, neste exemplo, tenhamos optado por usar o Google Colab pela conveniência e para demonstrar a exibição com Markdown, a migração para uma interface mais robusta pode ser feita com facilidade e oferecer uma experiência mais fluida para o usuário final caso deseje publicar essa aplicação.
